In [ ]:
# ─── Authentication & Imports ────────────────────────────────────────────────────
import ee
import geemap
import pandas
import ipywidgets as widgets
from datetime import date, timedelta
from IPython.display import display

# Authenticate & initialize Earth Engine
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# ─── 1. Map & Draw Control ────────────────────────────────────────────────────────
m = geemap.Map(center=(20, 0), zoom=2)
m.add_draw_control()

# ─── 2. Date Pickers ───────────────────────────────────────────────────────────────
today = date.today()
start_picker = widgets.DatePicker(description='Start', value=today - timedelta(days=30))
end_picker   = widgets.DatePicker(description='End',   value=today)

# ─── 3. Dataset Dropdown ──────────────────────────────────────────────────────────
var_dropdown = widgets.Dropdown(
    options=[('Sentinel‑2', 'S2'), ('Sentinel‑1', 'S1')],
    value='S2',
    description='Dataset:'
)

# ─── 4. Band Selector ─────────────────────────────────────────────────────────────
band_options_s2 = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12','RGB']
band_options_s1 = ['VV','VH']
band_selector = widgets.SelectMultiple(description='Bands:', rows=6)

def update_bands(change):
    if change['new'] == 'S2':
        band_selector.options = band_options_s2
        band_selector.value = ['B4','B3','B2']
    else:
        band_selector.options = band_options_s1
        band_selector.value = ['VV','VH']
var_dropdown.observe(update_bands, names='value')
update_bands({'new': var_dropdown.value})

# ─── 5. Export Logic ──────────────────────────────────────────────────────────────
export_button = widgets.Button(description="Export to Drive", button_style='success')
out = widgets.Output()

def on_export_clicked(btn):
    with out:
        out.clear_output()
        # 5.1 ROI
        if m.user_roi is None:
            print("▶️ Draw an AOI first."); return
        geom = m.user_roi

        # 5.2 Dates → EE
        start = ee.Date(str(start_picker.value))
        end   = ee.Date(str(end_picker.value)).advance(1, 'day')

        # 5.3 Bands chosen
        sel = list(band_selector.value)
        if not sel:
            print("▶️ Select at least one band."); return

        # 5.4 Build collection
        if var_dropdown.value == 'S2':
            col = (ee.ImageCollection('COPERNICUS/S2_SR')
                   .filterDate(start, end)
                   .filterBounds(geom)
                   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))
            scale = 10
        else:
            col = (ee.ImageCollection('COPERNICUS/S1_GRD')
                   .filterDate(start, end)
                   .filterBounds(geom)
                   .filter(ee.Filter.eq('instrumentMode', 'IW')))
            scale = 10

        # 5.5 Count & guard
        count = col.size().getInfo()
        print(f"ℹ️ {count} images found.")
        if count == 0:
            print("⚠️ No images—try a larger AOI or longer date range."); return

        # 5.6 Determine if user wants RGB composite
        want_rgb = 'RGB' in sel
        if want_rgb:
            sel.remove('RGB')

        # Preview: show RGB or the first selected band as a sanity check
        preview_bands = ['B4','B3','B2'] if want_rgb else [sel[0]]
        vis = ({'bands': preview_bands, 'min':0,'max':3000}
               if var_dropdown.value=='S2'
               else {'bands':preview_bands,'min':-25,'max':5})
        comp_preview = col.select(preview_bands).median().clip(geom)
        m.center_object(geom, 10)
        m.addLayer(comp_preview, vis, 'Preview')

        # 5.7 Export tasks: one per band (plus one RGB if requested)
        tasks_launched = 0

        # RGB composite first
        if want_rgb and var_dropdown.value=='S2':
            img = col.select(['B4','B3','B2']).median().clip(geom)
            desc = 'S2_RGB_composite'
            ee.batch.Export.image.toDrive(
                image=img, description=desc,
                folder='EarthEngineExports',
                fileNamePrefix=desc,
                region=geom, scale=scale,
                maxPixels=1e13
            ).start()
            print(f"✅ Started export: {desc}")
            tasks_launched += 1

        # Individual band exports
        for band in sel:
            img = col.select([band]).median().clip(geom)
            prefix = f"{var_dropdown.value}_{band}"
            ee.batch.Export.image.toDrive(
                image=img, description=prefix,
                folder='EarthEngineExports',
                fileNamePrefix=prefix,
                region=geom, scale=scale,
                maxPixels=1e13
            ).start()
            print(f"✅ Started export: {prefix}")
            tasks_launched += 1

        print(f"▶️ {tasks_launched} export task(s) launched. Monitor at https://code.earthengine.google.com/tasks")

# Wire up & display
export_button.on_click(on_export_clicked)
controls = widgets.VBox([
    widgets.HBox([start_picker, end_picker]),
    var_dropdown,
    band_selector,
    export_button,
    out
])
display(controls, m)


: 